In [1]:
import os

os.environ['OPENAI_API_KEY'] = ""

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from IPython.display import Markdown, display

In [8]:
import os
import subprocess

# specify url
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"

# specify dir
download_dir = "data"

# specify name
new_filename = "attention-is-all-you-need.pdf"

# create directory if it does not exist
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# use wget to download the PDF file into the directory
command = f"wget -P {download_dir} {pdf_url}"

# execute the wget command using subprocess
try:
    subprocess.run(command, shell=True, check=True)
    print(f"PDF downloaded successfully to {download_dir}")

    # Rename the downloaded file to the desired name
    old_filepath = os.path.join(download_dir, os.path.basename(pdf_url))
    new_filepath = os.path.join(download_dir, new_filename)
    os.rename(old_filepath, new_filepath)
    print(f"PDF renamed to {new_filename}")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

--2024-04-24 17:58:41--  https://arxiv.org/pdf/1706.03762.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 

PDF downloaded successfully to data
PDF renamed to attention-is-all-you-need.pdf


200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘data/1706.03762.pdf’

     0K .......... .......... .......... .......... ..........  2% 1.50M 1s
    50K .......... .......... .......... .......... ..........  4% 2.71M 1s
   100K .......... .......... .......... .......... ..........  6% 3.91M 1s
   150K .......... .......... .......... .......... ..........  9% 42.7M 1s
   200K .......... .......... .......... .......... .......... 11% 4.05M 1s
   250K .......... .......... .......... .......... .......... 13% 3.76M 1s
   300K .......... .......... .......... .......... .......... 16% 14.5M 0s
   350K .......... .......... .......... .......... .......... 18% 6.18M 0s
   400K .......... .......... .......... .......... .......... 20% 51.0M 0s
   450K .......... .......... .......... .......... .......... 23%  228M 0s
   500K .......... .......... .......... .......... .......... 25%  159M 0s
   550K .......... .......... .......... .......... .......... 27% 5.25M 0s
   600

In [9]:
!pip install pypdf

# Load documents and build index

In [10]:
# document loader
documents = SimpleDirectoryReader("data").load_data()

In [11]:
len(documents)

15

In [12]:
documents[0]

Document(id_='1420c07f-ffd7-44e1-85ec-070b6a9650f0', embedding=None, metadata={'page_label': '1', 'file_name': 'attention-is-all-you-need.pdf', 'file_path': '/Users/avivnur/code/cs573/grad_final/cqa-project/data/attention-is-all-you-need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob

In [13]:
# build index
index = VectorStoreIndex.from_documents(documents)

# Query

In [14]:
query_engine = index.as_query_engine()

In [15]:
response = query_engine.query("What is the paper about?")

In [16]:
response

Response(response='The paper is about a model called "attention is all you need" that focuses on attention mechanisms in neural networks, particularly in the context of natural language processing tasks.', source_nodes=[NodeWithScore(node=TextNode(id_='27af2aab-c09a-4a86-9886-e1e424648e31', embedding=None, metadata={'page_label': '15', 'file_name': 'attention-is-all-you-need.pdf', 'file_path': '/Users/avivnur/code/cs573/grad_final/cqa-project/data/attention-is-all-you-need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-04-24', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b6d69263-9152-4d20-a477-1905d9a4b279', node_type=<ObjectType.D

In [17]:
display(Markdown(f"<b>{response}</b>"))

<b>The paper is about a model called "attention is all you need" that focuses on attention mechanisms in neural networks, particularly in the context of natural language processing tasks.</b>

# Storing and Loading the index

In [18]:
index.storage_context.persist()

In [19]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context=storage_context)

# How to customize it

In [21]:
from llama_index.core import ServiceContext, set_global_service_context
from llama_index.llms.openai import OpenAI

# define LLM
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

# configure service context
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=500, chunk_overlap=50)

# set_global_service_context
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

/var/folders/lx/t6qp34dx07s89w02jy02_f8c0000gp/T/ipykernel_35844/2272916476.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, chunk_size=500, chunk_overlap=50)


In [22]:
query_engine = index.as_query_engine(streaming=True)
response = query_engine.query("What is the paper about ?")
response.print_response_stream()

The paper is about a study on attention mechanisms in neural networks, specifically focusing on the behavior of attention heads in the encoder self-attention at layer 5 of 6. The study examines how different attention heads learn to perform various tasks related to sentence structure, such as anaphora resolution.